In [2]:
print("manish")

manish


In [35]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\manis\anaconda3\envs\mchatbot\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [24]:
PINECONE_API_KEY="pcsk_7KRq2a_DBuYTodKjiFoChiAHCQAU5WKjVPs6s1pWfY9UyagocMdmv1yVitqMWcRXGmKy5W"
PINECONE_API_KEY="us-east-1"

In [10]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [18]:
# Import the necessary loaders from langchain
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

def load_pdf(directory_path):
    # Initialize DirectoryLoader with the directory and PyPDFLoader to handle PDF files
    loader = DirectoryLoader(directory_path, 
                             glob="*.pdf", 
                             loader_cls=PyPDFLoader)

    # Load all documents (PDFs) from the directory
    documents = loader.load()
    return documents

# Example usage
extracted_data = load_pdf("data/")  # Replace "data/" with the directory where your PDFs are stored
print(extracted_data)


Ignoring wrong pointing object 606 0 (offset 2164461)
Object 606 0 not defined.
 impossible to decode XFormObject /Im80


[Document(metadata={'source': 'data\\design_of_reinforced.pdf', 'page': 0}, page_content=''), Document(metadata={'source': 'data\\design_of_reinforced.pdf', 'page': 1}, page_content=''), Document(metadata={'source': 'data\\design_of_reinforced.pdf', 'page': 2}, page_content='Design of\nReinforced Concrete'), Document(metadata={'source': 'data\\design_of_reinforced.pdf', 'page': 3}, page_content=''), Document(metadata={'source': 'data\\design_of_reinforced.pdf', 'page': 4}, page_content='NINTH\nEDITION\nDesign of\nReinforced Concrete\nACI 318-11 Code Edition\nJack C. McCormac\nClemson University\nRussell H. Brown\nClemson University'), Document(metadata={'source': 'data\\design_of_reinforced.pdf', 'page': 5}, page_content='VP & EXECUTIVE PUBLISHER Don Fowley\nMARKETING MANAGER Christ opher Ruel\nACQUISITIONS EDITOR Jennifer Welter\nSENIOR PRODUCTION EDITOR Sujin Hong\nCREATIVE DIRECTOR Harry Nolan\nSENIOR DESIGNER Thomas Nery\nPHOTO EDITOR Sheena Goldstein\nCOVER PHOTO Frank Leung/iStoc

In [19]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [23]:
# Import RecursiveCharacterTextSplitter from langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(extracted_data):
    # Initialize the text splitter with the desired chunk size and overlap
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    
    # Split the text into chunks
    text_chunks = text_splitter.split_text(extracted_data)
    
    return text_chunks

# Example usage
extracted_data = "This is an example of text that needs to be split into chunks."
text_chunks = text_split(extracted_data)
print("Length of my chunk:", len(text_chunks))


Length of my chunk: 1


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    # Indented correctly
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [4]:
embeddings = download_hugging_face_embeddings()

C:\Users\manis\AppData\Local\Temp\ipykernel_3324\1660205919.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\manis\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\manis\anaconda3\envs\mchatbot\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated fil

In [5]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [26]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="negi"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

NameError: name 'text_chunks' is not defined